In [1]:
from numpy.lib.stride_tricks import as_strided
import numpy as np

In [ ]:
k_diags = np.array([0.25 * np.array([[-1, 0, 0, 0],
                                    [0, 1, 0, 0],
                                    [0, 0, 1, 0],
                                    [0, 0, 0, -1]]),
                    0.25 * np.array([[0, 0, 0, -1],
                                    [0, 0, 1, 0],
                                    [0, 1, 0, 0],
                                    [-1, 0, 0, 0]])])

k_lines = [0.25 * np.array([[-1, 1, 1, -1]]), 0.25 * np.array([[-1],[1],[1],[-1]])]

In [ ]:
arr = np.zeros((10, 10))

In [ ]:
arr[1][2] = 1
arr[2][2] = -1
arr[3][2] = -1
arr[4][2] = 1

arr[5][5] = 1
arr[5][6] = -1
arr[5][7] = -1
arr[5][8] = 1

arr[6][0] = 1
arr[7][1] = -1
arr[8][2] = -1
arr[9][3] = 1


arr

In [ ]:
sub_shape = k_diags[0].shape
print(sub_shape)

In [ ]:
view_shape = tuple(np.subtract(arr.shape, sub_shape) + 1) + sub_shape

In [ ]:
np.subtract(arr.shape, sub_shape) + 1

In [ ]:
view_shape

In [ ]:
arr_view = as_strided(arr, view_shape, arr.strides * 2)

In [ ]:
arr.strides * 2

In [ ]:
arr_view

In [ ]:
arr_view = arr_view.reshape((-1,) + sub_shape)
arr_view

In [ ]:
res = np.multiply(arr_view, k_diags[0])

In [ ]:
res

In [ ]:
res.shape

In [ ]:
np.einsum('ijk->i', res)

In [4]:
arr_test = np.zeros((10,10))

In [5]:
arr_test[2][8] = 2

In [6]:
arr_res = np.argwhere(arr_test > 3)

In [7]:
arr_res.shape[0] == 0

True

In [8]:
arr_res[0]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [9]:
-1 * 0.25 * np.array([-1, 1, 1, -1])

array([ 0.25, -0.25, -0.25,  0.25])

In [10]:
k_vertical = 0.25 * np.array([[-1], [1], [1], [-1]])
k_horizontal = 0.25 * np.array([[-1], [1], [1], [-1]])

In [11]:
arr_test[0][1] = 1
arr_test[1][1] = -1
arr_test[2][1] = -1
arr_test[3][1] = 1
arr_test

array([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [2]:
k_diags = np.array([0.25 * np.array([[-1, 0, 0, 0],
                                    [0, 1, 0, 0],
                                    [0, 0, 1, 0],
                                    [0, 0, 0, -1]]),
                    0.25 * np.array([[0, 0, 0, -1],
                                    [0, 0, 1, 0],
                                    [0, 1, 0, 0],
                                    [-1, 0, 0, 0]])])

k_lines = [0.25 * np.array([[-1, 1, 1, -1]]), 0.25 * np.array([[-1],[1],[1],[-1]])]

def _subboard_4_Conv2D(grid, k_shape:tuple, stride:tuple) -> np.array:
    """ Generates the sub view of the grid to be multiply with the kernel.
    First the shape of the sub_grid array is calculated, it depends on
    the grid shape and the kernel shape.
    The sub_grid array shape will be (n_x, n_y, k_x, k_y) with:
        * n_x: number of application of the kernel along row (with stride of 1)
        * n_y: number of application of the kernel along column (with stride of 1)
        * k_x, k_y: the shape of the kernel
    In this way sub_grid is a numpy array of n_x/n_y rows/columns of (k_x x k_y)
    sub view of the grid.
    Args:
    -----
        k_shape ([tuple[int]]): shape of the kernel
        stride ([tuple(int)]): put self.grid.strides * 2 (but why?)
    """
    view_shape = tuple(np.subtract(grid.shape, k_shape) + 1) + k_shape
    sub_grid = as_strided(grid, view_shape, stride * 2)
    return sub_grid


def _my_conv2D(grid, kernel:np.array) -> np.array:
    """ Retrieves the sub_grid from the function _subboard_4_Conv2D and performs
    the convolution (array multiplication + einstein sum along the 3rd and 4th
    dimensions).
    Args:
    -----
        * kernel ([np.array]): the kernel to use for convolution.
    """
    sub_grid = _subboard_4_Conv2D(grid, k_shape=kernel.shape, stride=grid.strides)
    res_conv = np.multiply(sub_grid, kernel)
    convolved = np.einsum('ijkl->ij', res_conv)
    return convolved.astype('int8')


def check_board(grid):
    """[summary]
    """
    ## Checking if white pair captured
    # Checking the diagonal:
    conv_diag1 = _my_conv2D(k_diags[0])
    conv_diag2 = _my_conv2D(k_diags[1])
    # Checking vertical and horizontal
    conv_lin1 = _my_conv2D(k_lines[0])
    conv_lin2 = _my_conv2D(k_lines[1])
    
    coord_cd1 = np.argwhere(conv_diag1 == 1)
    coord_cd2 = np.argwhere(conv_diag2 == 1)
    coord_cl1 = np.argwhere(conv_lin1 == 1)
    coord_cl2 = np.argwhere(conv_lin2 == 1)
    if coord_cd1.shape[0] != 0:
        for coord in coord_cd1:
            print("[check_board] - conv_diag1")
            grid[coord[0] + 1][coord[1] + 1] = 0
            grid[coord[0] + 2][coord[1] + 2] = 0
    if coord_cd2.shape[0] != 0:
        for coord in coord_cd2:
            print("[check_board] - conv_diag2")
            grid[coord[0] + 1][coord[1] + 2] = 0
            grid[coord[0] + 2][coord[1] + 1] = 0
    if coord_cl1.shape[0] != 0:
        for coord in coord_cl1:
            print("[check_board] - conv_lin1")
            grid[coord[0]][coord[1] + 1] = 0
            grid[coord[0]][coord[1] + 2] = 0
    if coord_cl2.shape[0] != 0:
        for coord in coord_cl2:
            print("[check_board] - conv_lin2")
            grid[coord[0] + 1][coord[1]] = 0
            grid[coord[0] + 2][coord[1]] = 0
    ## Checking if black pair captured
    # Checking the diagonal:
    conv_diag1 = _my_conv2D(-1 * k_diags[0])
    conv_diag2 = _my_conv2D(-1 * k_diags[1])
    # Checking vertical and horizontal
    conv_lin1 = _my_conv2D(-1 * k_lines[0])
    conv_lin2 = _my_conv2D(-1 * k_lines[1])
    print("<<<== CONV_LIN1 ==>>>")
    print(conv_lin1)
    print("<<<== CONV_LIN2 ==>>>")
    print(conv_lin2)
    print("<<<== ========= ==>>>")
    coord_cd1 = np.argwhere(conv_diag1 == 1)
    coord_cd2 = np.argwhere(conv_diag2 == 1)
    coord_cl1 = np.argwhere(conv_lin1 == 1)
    coord_cl2 = np.argwhere(conv_lin2 == 1)
    if coord_cd1.shape[0] != 0:
        for coord in coord_cd1:
            print("[check_board] - conv_diag1 -1")
            grid[coord[0] + 1][coord[1] + 1] = 0
            grid[coord[0] + 2][coord[1] + 2] = 0
    if coord_cd2.shape[0] != 0:
        for coord in coord_cd2:
            print("[check_board] - conv_diag2 -1")
            grid[coord[0] + 1][coord[1] + 2] = 0
            grid[coord[0] + 2][coord[1] + 1] = 0
    if coord_cl1.shape[0] != 0:
        for coord in coord_cl1:
            print("[check_board] - conv_lin1 -1")
            grid[coord[0]][coord[1] + 1] = 0
            grid[coord[0]][coord[1] + 2] = 0
    if coord_cl2.shape[0] != 0:
        for coord in coord_cl2:
            print("[check_board] - conv_lin2 -1")
            grid[coord[0] + 1][coord[1]] = 0
            grid[coord[0] + 2][coord[1]] = 0

In [ ]:
_subboard_4_Conv2D(arr_test, k_vertical, arr_test.strides)

In [ ]:
view_shape = tuple(np.subtract(arr_test.shape, k_vertical.shape) + 1) + k_vertical.shape

In [ ]:
view_shape

In [ ]:
sub_grid = as_strided(arr_test, view_shape, arr_test.strides * 2)


In [ ]:
sub_grid

In [ ]:
res_conv = np.multiply(sub_grid, k_vertical)


In [ ]:
res_conv

In [ ]:
convolved = np.einsum('ijkl->kl', res_conv)

In [ ]:
convolved

In [90]:
def issimplefreethree_position(yx, grid):
    """[summary]
    Args:
    yx ([type]): [description]
    grid ([type]): [description]
    Returns:
    ___: [description]
    """
    tmp = np.zeros((18,18))
    tmp[4:-4, 4:-4] = grid + 1 
    r_start, r_end = yx[0], yx[0] + 9
    c_start, c_end = yx[1], yx[1] + 9
    print(f"r_start = {r_start} -- r_end = {r_end}")
    print(f"c_start = {c_start} -- c_end = {c_end}")
    print(tmp[r_start:r_end, c_start:c_end])
    res = [_my_conv2D(tmp[r_start:r_end, c_start:c_end], kernel) for kernel in k_free_threes]
    if any([np.any(arr == 16) for arr in res]):
        return True, res
    return False, res
            

In [278]:
def isdoublefreethree_position(yx, grid) -> bool:
    """[summary]
    Args:
        yx ([type]): [description]
        grid ([type]): [description]
    Returns:
        bool: [description]
    """
    tmp = np.zeros((18,18))
    tmp[4:-4, 4:-4] = grid + 1 
    r_start, r_end = yx[0], yx[0] + 9
    c_start, c_end = yx[1], yx[1] + 9
    tmp[yx[0] + 4, yx[1] + 4] = 2
            
    print(tmp[r_start:r_end, c_start:c_end])
    res = (_my_conv2D(tmp[r_start:r_end, c_start:c_end], kernel) for kernel in k_free_threes)
    positions = (np.argwhere(arr >= 16) for arr in res)
    positions = [pos for pos in positions if pos.shape[0] != 0]
    if len(positions) > 0:
        print(positions)
        [print(pos.shape) for pos in positions]
        coords = np.hstack(positions)
        return True, res, positions
    return False, res, positions

In [279]:
grid1 = np.zeros((10,10))
grid2 = np.zeros((10,10))
grid3 = np.zeros((10,10))
grid4 = np.zeros((10,10))
grid5 = np.zeros((10,10))
grid6 = np.zeros((10,10))
grid7 = np.zeros((10,10))
grid8 = np.zeros((10,10))
grid9 = np.zeros((10,10))
grid10 = np.zeros((10,10))

In [280]:
#k_free_threes = [np.ones((1,6)), np.ones((6,1)), np.identity(6), np.rot90(np.identity(6))]
k_free_threes = [np.array([[1, 2, 2, 2, 2, 1]]),
                 np.array([[1], [2], [2], [2], [2], [1]]),
                 np.array([[1, 0, 0, 0, 0, 0],
                           [0, 2, 0, 0, 0, 0],
                           [0, 0, 2, 0, 0, 0],
                           [0, 0, 0, 2, 0, 0],
                           [0, 0, 0, 0, 2, 0],
                           [0, 0, 0, 0, 0, 1]]),
                 np.array([[0, 0, 0, 0, 0, 1],
                           [0, 0, 0, 0, 2, 0],
                           [0, 0, 0, 2, 0, 0],
                           [0, 0, 2, 0, 0, 0],
                           [0, 2, 0, 0, 0, 0],
                           [1, 0, 0, 0, 0, 0]])]

In [281]:
grid1[0:3,2] = 1

grid2[1:4,2] = 1

grid3[5,0:3] = 1

grid4[6,2:5] = 1

grid5[2,2] = 1
grid5[3,3] = 1
grid5[4,4] = 1

grid6[2][2] = 1
grid6[4][4] = 1
grid6[5][5] = 1

grid7[2][2] = 1
grid7[3][3] = 1
grid7[5][5] = 1

grid8[[6, 7, 8],[6, 7, 8]] = 1
grid9[[2, 3, 5, 5], [2, 3, 6, 7]] = 1
grid10[[1, 1, 1, 1],[1, 2, 5, 6]] = 1

In [282]:
grid9

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [283]:
b, res, positions = isdoublefreethree_position([5,5], grid9)

[[1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 2. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 2. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 2. 2. 2. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1.]]
[array([[4, 2],
       [4, 3]]), array([[0, 0]])]


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 2 and the array at index 1 has size 1

In [263]:
res

[array([[ 9,  7,  5,  3],
        [11,  8,  5,  3],
        [11,  9,  6,  3],
        [11,  9,  7,  4],
        [11,  9,  7,  5],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0]], dtype=int8),
 array([[ 9, 11, 11, 11, 11,  0,  0,  0,  0],
        [ 7,  8,  9,  9,  9,  0,  0,  0,  0],
        [ 5,  5,  6,  7,  7,  0,  0,  0,  0],
        [ 3,  3,  3,  4,  5,  0,  0,  0,  0]], dtype=int8),
 array([[17,  7,  5,  3],
        [ 7, 14,  5,  3],
        [ 5,  5, 10,  3],
        [ 3,  3,  3,  6]], dtype=int8),
 array([[8, 8, 4, 4],
        [8, 4, 4, 0],
        [4, 4, 0, 0],
        [4, 0, 0, 0]], dtype=int8)]

In [185]:
test = np.array([[]])


In [187]:
np.vstack((res)).shape

(3, 2)

In [29]:
yx =[6,1]
tmp = np.zeros((18,18))
tmp[4:-4, 4:-4] = grid4 + 1 
r_start, r_end = yx[0], yx[0] + 8
c_start, c_end = yx[1], yx[1] + 8
_my_conv2D(tmp[r_start:r_end, c_start:c_end], k_free_threes[0])

array([[ 5,  7,  9],
       [ 5,  7,  9],
       [ 5,  7,  9],
       [ 5,  7,  9],
       [ 6, 10, 14],
       [ 5,  7,  9],
       [ 5,  7,  9],
       [ 5,  7,  9]], dtype=int8)

In [30]:
k_free_threes[0]

array([[1, 2, 2, 2, 2, 1]])